In [1]:
import pandas as pd
import numpy as np
from RidgeOrdinalRegressor import RidgeOrdinalRegressor
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse

In [2]:
# this is from the examples in R's ordinal package
# they fit a weighted model; just repeat observations for simplicity for now
df = pd.read_csv('housing.csv')
n_obs = df['Freq'].sum()
df.shape[0],n_obs

(72, 1681)

In [3]:
df = df.loc[np.repeat(df.index, df['Freq']), :].drop('Freq', axis='columns')
df.sample(n=3)

,Sat,Infl,Type,Cont
12,Low,Medium,Apartment,Low
37,Medium,Low,Tower,High
46,Medium,Low,Apartment,High


`fm1 <- clm(Sat ~ Infl + Type + Cont, weights = Freq, data = housing)`

In [4]:
y = pd.get_dummies(df['Sat'])[['Low', 'Medium', 'High']].values
y.shape

(1681, 3)

In [5]:
# Infl has 3 possible values, Type: 4, Cont: 2
# so there should be 3 + 4 + 2 -3 = 6 columns
X = pd.get_dummies(df[['Infl', 'Type', 'Cont']], drop_first=True)
X = sparse.csr_matrix(X.values)
assert X.shape[1] == 6
assert X.shape[0] == n_obs

In [6]:
rr = RidgeOrdinalRegressor(alpha=0, tol=1e-30)
rr.fit(X, y)
rr.intercept_, rr.coef_

      fun: 1739.5746495294743
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.19866894e-06,  4.04623847e-07, -4.53114495e-07,  2.20956383e-06,
        1.74848037e-06,  2.47647002e-07,  3.46821878e-08, -4.38696617e-07])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 35
      nit: 27
   status: 0
  success: True
        x: array([-1.57288824, -0.38604484,  1.2888191 ,  0.72242537, -0.20616361,
        0.51866466, -0.57235   ,  0.36028399])


(array([-1.57288824, -0.38604484]),
 array([ 1.2888191 ,  0.72242537, -0.20616361,  0.51866466, -0.57235   ,
         0.36028399]))

In [7]:
R_intercepts = np.array([-0.4961351, 0.6907083])
assert np.allclose(rr.intercept_, R_intercepts)

AssertionError: 

In [8]:
R_coefs = np.array([0.5663937, 1.2888191, -0.5723500,
   -0.3661864,    -1.0910147,     0.3602840])
assert np.allclose(rr.coef_, R_coefs)

AssertionError: 

In [9]:
# from https://gist.github.com/dsaxton/92baf15b74c859e714a83f09029bf5b4
def negloglik(X, y, params):
    """
    First elements of params are eta terms, remaining 
    are beta coefficients.
    """
    alpha = params[:(y.shape[1]-1)]
    beta = params[(y.shape[1]-1):]
    #alpha = np.append(eta[0], np.exp(eta[1:])).cumsum()
    
    s = np.dot(X, beta).reshape(-1, 1)
    dfunc = 1 / (np.exp(-(
        np.repeat(s, len(alpha), axis=1) + alpha)) + 1)
    dfunc = np.append(np.zeros(dfunc.shape[0])[:, np.newaxis], 
                      np.append(dfunc, 
                                np.ones(dfunc.shape[0])[:, np.newaxis], 
                                axis=1), 
                      axis=1)
    
    mass = np.diff(dfunc, axis=1)    
    return - np.sum(np.log(np.sum(np.multiply(mass, y), axis=1)))

In [10]:
negloglik(X=X.todense(), y=y, params=np.concatenate([rr.intercept_, rr.coef_]))

1739.5746495294743

In [13]:
from scipy.optimize import minimize
Xdense = X.todense()

def nll(params):
    return negloglik(Xdense, y, params)

res = minimize(fun=nll, x0=np.random.randn(X.shape[1] + y.shape[1] - 1), tol=1e-6)
res['x']

array([-1.57288834, -0.38604494,  1.28881924,  0.72242547, -0.20616369,
        0.51866467, -0.57235   ,  0.360284  ])

In [14]:
assert np.allclose(np.concatenate([rr.intercept_, rr.coef_]), res['x'])